In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from gurobipy import *

In [100]:
def ap(demand,costs,startendparams,hourparams):
    
    Dt = demand
    Dt = np.insert(Dt,0,0)
    numperiods = len(Dt)-1
    period = np.arange(1,numperiods+1)
    cm,ci,cb,cf,ch,cw,co,cc = costs
    W0,Wtmax,Wtmin,I0,Itmin,B0,Btmax = startendparams
    L,H,M = hourparams

    model = Model('AP')
    Ht,Ft,Wt,It,Ot,Bt,Pt,Ct = {},{},{},{},{},{},{},{}

    It[0] = I0
    Wt[0] = W0
    Bt[0] = B0

    for p in period:
        It[p] = model.addVar(vtype="I", name="I(%s)"%p)
        Wt[p] = model.addVar(vtype="I",name="W(%s)"%p)
        Ht[p] = model.addVar(vtype="I", name="H(%s)"%p)
        Ft[p] = model.addVar(vtype="I", name="F(%s)"%p)
        Ot[p] = model.addVar(vtype="I", name="O(%s)"%p)
        Bt[p] = model.addVar(vtype="I",name="B(%s)"%p)
        Pt[p] = model.addVar(vtype="I", name="P(%s)"%p)
        Ct[p] = model.addVar(vtype="I",name="C(%s)"%p)



    model.update()
    for p in period:
        model.addConstr((Pt[p]-((H*Wt[p]+Ot[p])/L))<=0,"Link1(%s)"%p)
        model.addConstr((Wt[p]-Wt[p-1]-Ht[p]+Ft[p])==0,"Link2(%s)"%p)
        model.addConstr((It[p]-It[p-1]-Pt[p]-Ct[p]+Dt[p]-Bt[p]+Bt[p-1])==0,"Link3(%s)"%p)
        model.addConstr((Ot[p]-M*Wt[p])<=0,"Link4(%s)"%p)

    model.addConstr((Bt[numperiods]<=Btmax),"EndingBackLog")
    model.addConstr((Wt[numperiods]>=Wtmin),"EndingWorkforceMin")
    model.addConstr((Wt[numperiods]<=Wtmax),"EndingWorkforceMax")
    model.addConstr((It[numperiods]>=Itmin),"EndingInventory")

    reglabor_cost = quicksum(Wt[p] for p in period)*cw
    otlabor_cost = quicksum(Ot[p] for p in period)*co
    cost_hiring = quicksum(Ht[p] for p in period)*ch
    cost_firing = quicksum(Ft[p] for p in period)*cf
    cost_inv = quicksum(It[p] for p in period)*ci
    cost_so = quicksum(Bt[p] for p in period)*cb
    cost_mat = quicksum(Pt[p] for p in period)*cm
    cost_os = quicksum(Ct[p] for p in period)*cc
    
    total_cost = reglabor_cost+otlabor_cost+cost_hiring+cost_firing+cost_inv+cost_so+cost_mat+cost_os
    model.setObjective(total_cost,
        GRB.MINIMIZE)
    model.update()
    model.__data = costlist,total_cost
    return model

In [243]:
demand = np.array([2800,2800,1000,920,780,950,1050,1200,2000,2500,3000,2800])
cm = 75
ci = 25
cb = 50
cf = 999999
ch = 1200
cw = 2400
co = 9999
cc = 9999
L = 3.5
H = 160
M = 10
W0 = 36
Wtmax = 36
Wtmin = 30
I0 = 500
Itmin = 500
B0 = 0
Btmax = 0
costs = np.array([cm,ci,cb,cf,ch,cw,co,cc])
startendparams = np.array([W0,Wtmax,Wtmin,I0,Itmin,B0,Btmax])
hourparams = np.array([L,H,M])

model = ap(demand,costs,startendparams,hourparams)
model.optimize()


Optimize a model with 52 rows, 96 columns and 181 nonzeros
Variable types: 0 continuous, 96 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+01]
  Objective range  [3e+01, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+03]
Found heuristic solution: objective 2.540987e+08
Presolve removed 4 rows and 25 columns
Presolve time: 0.00s
Presolved: 48 rows, 71 columns, 152 nonzeros
Variable types: 0 continuous, 71 integer (0 binary)

Root relaxation: objective 7.465836e+06, 49 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7465836.18    0   35 2.5410e+08 7465836.18  97.1%     -    0s
H    0     0                    8930917.0000 7465836.18  16.4%     -    0s
H    0     0                    8732511.0000 7465836.18  14.5%     -    0s
H    0     0                    7846824.0000 7465836.18  4.86%     -    0s
H

In [244]:
df = pd.DataFrame({})
df['Time Period'] = np.arange(1,len(demand)+1)
df.set_index('Time Period',inplace=True)
myvars=model.getVars()
i = 0
p = 1
while(p<=12):

    df.loc[p,'Inventory (#)'] = myvars[i].X
    df.loc[p,'Workforce'] = myvars[i+1].X
    df.loc[p,'Employees Hired'] = myvars[i+2].X
    df.loc[p,'Employees Fired'] = myvars[i+3].X
    df.loc[p,'Overtime(hrs)'] = myvars[i+4].X
    df.loc[p,'Backlogged (units)'] = myvars[i+5].X
    df.loc[p,'Internal Production (units)'] = myvars[i+6].X
    df.loc[p,'Outsourced Production (units)'] = myvars[i+7].X
    p+=1
    i+=8

if ('sum' not in df.index):
    df = df.append(df.agg(['sum', 'mean']))


In [245]:
df

,Inventory (#),Workforce,Employees Hired,Employees Fired,Overtime(hrs),Backlogged (units),Internal Production (units),Outsourced Production (units)
1,-0.0,40.000000,4.000000,-0.000000,-0.0,472.0,1828.00,0.000000
2,-0.0,40.000000,0.000000,-0.000000,-0.0,1444.0,1828.00,0.000000
3,-0.0,40.000000,0.000000,-0.000000,-0.0,616.0,1828.00,0.000000
4,292.0,40.000000,0.000000,-0.000000,-0.0,-0.0,1828.00,0.000000
5,1340.0,40.000000,0.000000,-0.000000,-0.0,-0.0,1828.00,0.000000
6,2218.0,40.000000,0.000000,-0.000000,-0.0,-0.0,1828.00,0.000000
7,2996.0,40.000000,0.000000,-0.000000,-0.0,-0.0,1828.00,0.000000
8,3624.0,40.000000,0.000000,-0.000000,-0.0,-0.0,1828.00,0.000000
9,3452.0,40.000000,0.000000,0.000000,-0.0,-0.0,1828.00,0.000000
10,2780.0,40.000000,0.000000,0.000000,-0.0,-0.0,1828.00,0.000000


In [246]:
df2 = pd.DataFrame({})
df2.loc['Annual Category by Costs','Regular Labor Cost'] = df.loc['sum','Workforce']*cw
df2.loc['Annual Category by Costs','Overtime Labor Cost'] = df.loc['sum','Overtime(hrs)']*co
df2.loc['Annual Category by Costs','Cost of Hiring'] = df.loc['sum','Employees Hired']*ch
df2.loc['Annual Category by Costs','Cost of Firing'] = df.loc['sum','Employees Fired']*cf
df2.loc['Annual Category by Costs','Cost of Inventory'] = df.loc['sum','Inventory (#)']*ci
df2.loc['Annual Category by Costs','Cost of Stockouts'] = df.loc['sum','Backlogged (units)']*cb
df2.loc['Annual Category by Costs','Cost of Materials'] = df.loc['sum','Internal Production (units)']*cm
df2.loc['Annual Category by Costs','Cost of Outsourcing'] = df.loc['sum','Outsourced Production (units)']*cc

df2 = df2.transpose()
df2

,Annual Category by Costs
Regular Labor Cost,1142400.0
Overtime Labor Cost,0.0
Cost of Hiring,4800.0
Cost of Firing,3999996.0
Cost of Inventory,470250.0
Cost of Stockouts,126600.0
Cost of Materials,1631475.0
Cost of Outsourcing,469953.0


In [247]:
model.objVal

7845474.0